## 合并两个版本的 metadata 文件

In [11]:
import pandas as pd

# 配置
file1 = "/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/metadata_archive/Phyloseq_metadata.tsv"
file2 = "/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/metadata_archive/sampleTable_changed_250924_16S.tsv"
out_file = "/Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/metadata_archive/Phyloseq_ResMicroDb_metadata_merge.tsv"

prefix1 = "1_"   # 第一个文件的前缀
prefix2 = "2_"   # 第二个文件的前缀
key_col = "Run"  # 用来匹配的列名

# 读入两个 tsv
df1 = pd.read_csv(file1, sep="\t")
df2 = pd.read_csv(file2, sep="\t")

# 合并时，只给“两个表中都存在的同名列”加后缀
merged = pd.merge(df1, df2, on=key_col, how="inner", suffixes=("_1", "_2"))

# 原始列集合（除去 key）
cols1 = [c for c in df1.columns if c != key_col]
cols2 = [c for c in df2.columns if c != key_col]

set1 = set(cols1)
set2 = set(cols2)

common_cols = list(set1 & set2)  # 两个文件中都出现的列名

# 最终结果表
result = pd.DataFrame()
result[key_col] = merged[key_col]

identical_cols = []
different_cols = []

# 先按 df1 的列顺序处理，保证“同名列挨着放”
for col in cols1:
    if col in common_cols:
        # 这个列在两个文件中都有，对应 merged 里的 col_1 和 col_2
        col1 = f"{col}_1"
        col2 = f"{col}_2"

        s1 = merged[col1]
        s2 = merged[col2]

        if s1.equals(s2):
            # 内容完全一样：保留一列，用原名（不加前缀）
            result[col] = s1
            identical_cols.append(col)
        else:
            # 内容不一样：保留两列，加前缀，并挨在一起
            result[f"{prefix1}{col}"] = s1
            result[f"{prefix2}{col}"] = s2
            different_cols.append(col)
    else:
        # 只在 df1 中的列：合并后列名还是原名 col
        result[f"{prefix1}{col}"] = merged[col]

# 再处理只在 df2 中的列
for col in cols2:
    if col not in common_cols:
        # 只在 df2 中的列：合并后列名还是原名 col
        result[f"{prefix2}{col}"] = merged[col]

# 导出为 tsv
result.to_csv(out_file, sep="\t", index=False)

print(f"保存到 {out_file}")
print("完全相同且去重的列:", identical_cols)
print("同名但内容不同、已分别保留的列:", different_cols)


/var/folders/23/yxbb6h094616_yvzj1181f0m0000gn/T/ipykernel_24283/2104430228.py:13: DtypeWarning: Columns (11,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(file1, sep="\t")
/var/folders/23/yxbb6h094616_yvzj1181f0m0000gn/T/ipykernel_24283/2104430228.py:14: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(file2, sep="\t")


保存到 /Users/kiancai/STA24/CWD/STAi/MiCoGPT/data/try2_withCC/metadata_archive/Phyloseq_ResMicroDb_metadata_merge.tsv
完全相同且去重的列: ['Sequencing_Type', 'Library_Layout', 'Platform', 'Project_ID', 'BioSample', 'PMID', 'Reads', 'Observed']
同名但内容不同、已分别保留的列: ['Phenotype', 'Disease_Stage', 'Complication', 'Patient_ID', 'Sex', 'Antibiotics_Used', 'Time_Point', 'Age', 'age_start', 'age_end', 'BMI', 'Continent', 'Country', 'Location', 'Latitude', 'Longitude', 'Model', 'Age_Group', 'Shannon', 'Chao1']
